<a href="https://colab.research.google.com/github/GusBraud/mei_tools/blob/gesualdo_dev/meitools_Gesualdo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Update des métadonnées**

In [310]:
!rm -r mei_tools
!git clone https://github.com/GusBraud/mei_tools

Cloning into 'mei_tools'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 416 (delta 46), reused 32 (delta 15), pack-reused 342 (from 1)
Receiving objects: 100% (416/416), 730.32 KiB | 4.77 MiB/s, done.
Resolving deltas: 100% (229/229), done.


In [311]:
%cd mei_tools
%pip install -e .

/content/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools
Obtaining file:///content/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools/mei_tools
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mei_tools (pyproject.toml) ... done
  Created wheel for mei_tools: filename=mei_tools-2.0.2-py3-none-any.whl size=5254 sha256=f38dadc0d10086d496a860bc600

In [312]:
!pip install setuptools wheel twine

In [313]:
!python -c "import mei_tools; print('import successful')"

import successful


In [314]:
#  Import necessary libraries
import mei_tools
from mei_tools import MEI_Metadata_Updater
from mei_tools import MEI_Music_Feature_Processor
import glob
import os
import pandas as pd
from lxml import etree # Import etree here
import re # Import regex for cleaning editor names

# Modified MEI_Metadata_Updater class to add static encodingDesc, consolidate annotation text, and remove duplicate meiHeads
def apply_metadata_fixed(self, mei_file_path, metadata_dict, output_folder):
    """
    Adds or updates metadata in the first meiHead from a dictionary,
    consolidates annotation text and static encodingDesc content, adds xml:ids,
    and removes subsequent meiHeads.

    Args:
        mei_file_path (str): The path to the input MEI file.
        metadata_dict (dict): A dictionary containing the metadata to apply.
        output_folder (str): The folder to save the updated MEI file.
    """
    print(f"Getting {os.path.basename(mei_file_path)}")
    try:
        with open(mei_file_path, 'r', encoding='utf-8') as f:
            mei_content = f.read()

        # Use a parser to handle potential issues with the input XML
        parser = etree.XMLParser(remove_blank_text=True)
        root = etree.fromstring(mei_content.encode('utf-8'), parser=parser)
        ns = {'mei': 'http://www.music-encoding.org/ns/mei/5'}
        xml_ns = '{http://www.w3.org/XML/1998/namespace}' # Namespace for xml:id

        # Counter for generating unique xml:ids
        id_counter = 1

        def generate_xml_id(prefix="m"):
            nonlocal id_counter
            current_id = f"{prefix}-{id_counter}"
            id_counter += 1
            return current_id

        def generate_initials_id(name):
            """Generates an xml:id from the initials of a cleaned name."""
            # Remove trailing role information like " - Project manager"
            cleaned_name = re.sub(r'\s-\s.*$', '', name).strip()
            initials = ''.join([word[0].upper() for word in cleaned_name.split() if word])
            # Ensure the ID is a valid XML ID (starts with a letter or underscore, contains letters, digits, hyphens, underscores, periods, colons)
            # A simple approach is to just use the initials, assuming they are letters.
            # If more complex names/characters are expected, more robust validation/cleaning would be needed.
            return initials


        # Find all meiHead elements
        all_mei_heads = root.findall('.//meiHead', namespaces=ns)

        mei_head = None # Initialize mei_head to None
        mei_heads_to_remove = []


        if not all_mei_heads:
             print(f"Warning: No meiHead found in {os.path.basename(mei_file_path)}. Cannot update metadata.")
             # If no meiHead, we can't update.

        else:
            # Use the first meiHead found as the target (this is the one we populate with CSV data)
            mei_head = all_mei_heads[0]
            # Add xml:id to the first meiHead if it doesn't have one
            if xml_ns + 'id' not in mei_head.attrib:
                 mei_head.set(xml_ns + 'id', generate_xml_id())

            # Collect meiHeads to remove (all subsequent ones)
            mei_heads_to_remove.extend(all_mei_heads[1:])


            # --- Add the specific static encodingDesc block to the first meiHead ---
            # Define the static encodingDesc XML as a string
            encoding_desc_xml = """
<encodingDesc xml:id="m-5">
      <appInfo xml:id="m-6">
        <application xml:id="sibelius" isodate="2025-7-11T11:43:22Z" version="7130">
          <name xml:id="m-8" type="operating-system">Windows 7</name>
        </application>
        <application xml:id="sibmei" type="plugin" version="4.1.0">
          <name xml:id="m-10">Sibelius to MEI 4 Exporter (4.1.0)</name>
        </application>
      </appInfo>
    </encodingDesc>
"""
            # Parse the static XML snippet into an element tree
            parser_frag = etree.XMLParser(remove_blank_text=True)
            encoding_desc_elem = etree.fromstring(encoding_desc_xml.strip().encode('utf-8'), parser=parser_frag)

            # Find existing encodingDesc in the target meiHead
            existing_encoding_desc = mei_head.find('./encodingDesc', namespaces=ns)

            if existing_encoding_desc is not None:
                 # If encodingDesc exists, replace it with the new static one
                 existing_encoding_desc.getparent().replace(existing_encoding_desc, encoding_desc_elem)
                 print(f"  Replaced existing encodingDesc with the specified static block in {os.path.basename(mei_file_path)}")
            else:
                 # If encodingDesc doesn't exist, append the new static one to meiHead
                 # Find a position to insert it (e.g., after fileDesc if it exists)
                 file_desc_elem = mei_head.find('./fileDesc', namespaces=ns)
                 if file_desc_elem is not None:
                      # Find the index of fileDesc to insert encodingDesc after it
                      file_desc_index = mei_head.index(file_desc_elem)
                      mei_head.insert(file_desc_index + 1, encoding_desc_elem)
                 else:
                      # If no fileDesc, just append to meiHead
                      mei_head.append(encoding_desc_elem)

                 print(f"  Added specified static encodingDesc block to {os.path.basename(mei_file_path)}")


        # Proceed only if we have a valid mei_head (the first one found/updated)
        if mei_head is not None:
            # Find or create fileDesc within meiHead
            file_desc = mei_head.find('./fileDesc', namespaces=ns)
            if file_desc is None:
                file_desc = etree.SubElement(mei_head, "fileDesc")
                file_desc.set(xml_ns + 'id', generate_xml_id())

            # Find or create workList within meiHead
            work_list = mei_head.find('./workList', namespaces=ns)
            if work_list is None:
                 work_list = etree.SubElement(mei_head, "workList")
                 work_list.set(xml_ns + 'id', generate_xml_id())


            # Find the annot element created by the script (based on CSV data)
            target_annot = work_list.find('.//notesStmt/annot', namespaces=ns)

            # Ensure notesStmt and annot exist in the target workList if not already created by CSV data
            if target_annot is None:
                 notes_stmt = work_list.find('./notesStmt', namespaces=ns)
                 if notes_stmt is None:
                      notes_stmt = etree.SubElement(work_list, "notesStmt")
                      notes_stmt.set(xml_ns + 'id', generate_xml_id())
                 target_annot = etree.SubElement(notes_stmt, "annot")
                 target_annot.set(xml_ns + 'id', generate_xml_id())
                 target_annot.text = "" # Initialize with empty text


            # --- Consolidate annotation text from subsequent meiHeads into the target annot ---
            annotation_texts_to_merge = []
            all_mei_heads_for_merge = root.findall('.//meiHead', namespaces=ns) # Re-find all heads

            # Iterate through all meiHeads (except the target) to find annotations
            for extra_mei_head in all_mei_heads_for_merge:
                 if extra_mei_head is not None and extra_mei_head != mei_head: # Exclude the target meiHead
                     extra_annot = extra_mei_head.find('.//notesStmt/annot', namespaces=ns)
                     if extra_annot is not None and extra_annot.text:
                          annotation_texts_to_merge.append(extra_annot.text.strip())
                          print(f"  Extracted annotation text from extra meiHead for annotation consolidation in {os.path.basename(mei_file_path)}")


            # Append the collected annotation texts to the target annot
            if annotation_texts_to_merge:
                 existing_annot_text = target_annot.text.strip() if target_annot.text else ""
                 new_annot_text = existing_annot_text
                 if new_annot_text:
                     new_annot_text += " ; " # Add separator if existing text is present
                 new_annot_text += " ; ".join(annotation_texts_to_merge) # Join extracted texts with separator
                 target_annot.text = new_annot_text
                 print(f"  Consolidated annotation text in target annot.")


            # --- Update/Create metadata sections within fileDesc (using spreadsheet data) ---

            if file_desc is not None:
                # Find or create titleStmt
                title_stmt = file_desc.find('./titleStmt', namespaces=ns)
                if title_stmt is None:
                    title_stmt = etree.SubElement(file_desc, "titleStmt")
                    title_stmt.set(xml_ns + 'id', generate_xml_id())

                # Clear specific existing children within titleStmt to replace with metadata (overwriting original title/author if they were in first header)
                for child in list(title_stmt):
                     if child.tag.split('}')[-1] in ['title', 'author', 'composer', 'respStmt']:
                          title_stmt.remove(child)

                # Add title and author from metadata, with xml:ids
                title = etree.SubElement(title_stmt, "title")
                title.set(xml_ns + 'id', generate_xml_id())
                title.text = metadata_dict.get('Title', '')
                author = etree.SubElement(title_stmt, "author")
                author.set(xml_ns + 'id', generate_xml_id())
                author.text = metadata_dict.get('Composer_Name', '')


                # Find or create pubStmt - Preserve existing non-publisher content
                pub_stmt = file_desc.find('./pubStmt', namespaces=ns)
                if pub_stmt is None:
                    pub_stmt = etree.SubElement(file_desc, "pubStmt")
                    pub_stmt.set(xml_ns + 'id', generate_xml_id())

                # Clear *only* existing publisher tags added from metadata
                for pub in pub_stmt.findall('./publisher', namespaces=ns):
                     pub_stmt.remove(pub)

                # Add publisher information (Publisher_1 and Publisher_2) from metadata, with xml:ids
                if metadata_dict.get('Source_Publisher_1'):
                    publisher1 = etree.SubElement(pub_stmt, "publisher")
                    publisher1.set(xml_ns + 'id', generate_xml_id())
                    publisher1.text = metadata_dict.get('Source_Publisher_1', '')
                    if metadata_dict.get('Publisher_1_VIAF'):
                        publisher1.set('ref', metadata_dict.get('Publisher_1_VIAF', ''))
                if metadata_dict.get('Source_Publisher_2'):
                    publisher2 = etree.SubElement(pub_stmt, "publisher")
                    publisher2.set(xml_ns + 'id', generate_xml_id())
                    publisher2.text = metadata_dict.get('Source_Publisher_2', '')
                    if metadata_dict.get('Publisher_2_VIAF'):
                        publisher2.set('ref', metadata_dict.get('Publisher_2_VIAF', ''))


                # Find or create editionStmt - Clear existing editor structure
                edition_stmt = file_desc.find('./editionStmt', namespaces=ns)
                if edition_stmt is None:
                    edition_stmt = etree.SubElement(file_desc, "editionStmt")
                    edition_stmt.set(xml_ns + 'id', generate_xml_id())

                # Clear existing children within editionStmt (edition, respStmt)
                for child in list(edition_stmt):
                     if child.tag.split('}')[-1] in ['edition', 'respStmt']:
                          edition_stmt.remove(child)

                # Add edition and editor information if Editor data is present, with xml:ids (cleaned initials for persName)
                if metadata_dict.get('Editor'):
                     edition = etree.SubElement(edition_stmt, "edition")
                     edition.set(xml_ns + 'id', generate_xml_id())
                     resp_stmt = etree.SubElement(edition, "respStmt")
                     resp_stmt.set(xml_ns + 'id', generate_xml_id())
                     resp = etree.SubElement(resp_stmt, "resp")
                     resp.text = "editor"
                     editors = metadata_dict.get('Editor', '').split(' | ')
                     orcids = metadata_dict.get('Editor_ORCID', '').split(' | ')
                     editor_viafs = metadata_dict.get('Editor_VIAF', '').split(' | ')

                     for i, editor_name in enumerate(editors):
                         if editor_name.strip(): # Check if editor name is not empty
                             pers_name = etree.SubElement(resp_stmt, "persName")
                             # Generate xml:id from cleaned initials
                             pers_name_id = generate_initials_id(editor_name.strip())
                             pers_name.set(xml_ns + 'id', pers_name_id)
                             pers_name.text = editor_name.strip()
                             # Corrected auth attribute to "orcid"
                             if i < len(orcids) and orcids[i].strip():
                                  pers_name.set('auth', 'orcid')
                                  pers_name.set('auth.uri', orcids[i].strip()) # Use ORCID URI
                             elif i < len(editor_viafs) and editor_viafs[i].strip():
                                 pers_name.set('auth', 'VIAF') # Fallback to VIAF if no ORCID
                                 pers_name.set('auth.uri', editor_viafs[i].strip())
                             else:
                                 # If no ORCID or VIAF, set auth to "none" or omit auth/auth.uri
                                 pers_name.set('auth', 'none')


                # Find or create sourceDesc - Clear existing source structure
                source_desc = file_desc.find('./sourceDesc', namespaces=ns)
                if source_desc is None:
                    source_desc = etree.SubElement(file_desc, "sourceDesc")
                    source_desc.set(xml_ns + 'id', generate_xml_id())
                # If sourceDesc already exists, we don't add a new id here

                # Clear existing children within sourceDesc (source)
                for child in list(source_desc):
                     if child.tag.split('}')[-1] == 'source':
                          source_desc.remove(child)

                # Add source information, RDL_ID, and Permalink as bibl within sourceDesc/source, with xml:ids
                source = etree.SubElement(source_desc, "source")
                source.set(xml_ns + 'id', generate_xml_id())
                title_source = etree.SubElement(source, "title")
                title_source.set(xml_ns + 'id', generate_xml_id())
                title_source.text = metadata_dict.get('Source_Title', '')

                date_source = etree.SubElement(source, "date")
                date_source.set(xml_ns + 'id', generate_xml_id())
                date_source.text = str(metadata_dict.get('Source_Date', ''))

                country_source = etree.SubElement(source, "country")
                country_source.set(xml_ns + 'id', generate_xml_id())
                country_source.text = metadata_dict.get('Source_Country', '')

                settlement_source = etree.SubElement(source, "settlement")
                settlement_source.set(xml_ns + 'id', generate_xml_id())
                settlement_source.text = metadata_dict.get('Source_City', '')

                identifier_source = etree.SubElement(source, "identifier", type="shelfmark")
                identifier_source.set(xml_ns + 'id', generate_xml_id())
                identifier_source.text = metadata_dict.get('Source_Shelfmark', '')

                # Add RDL_ID and Permalink as bibl within sourceDesc/source, with xml:ids
                if metadata_dict.get('RDL_ID'):
                    rdl_id_elem = etree.SubElement(source, "identifier", type="RDL_ID")
                    rdl_id_elem.set(xml_ns + 'id', generate_xml_id())
                    rdl_id_elem.text = str(metadata_dict.get('RDL_ID', ''))

                if metadata_dict.get('Permalink'):
                    permalink_bibl = etree.SubElement(source, "bibl")
                    permalink_bibl.set(xml_ns + 'id', generate_xml_id())
                    permalink_bibl.text = metadata_dict.get('Permalink', '')


            # Find or create workList within meiHead
            work_list = mei_head.find('./workList', namespaces=ns)
            if work_list is None:
                 work_list = etree.SubElement(mei_head, "workList")
                 work_list.set(xml_ns + 'id', generate_xml_id())


            # Find the annot element created by the script (based on CSV data)
            target_annot = work_list.find('.//notesStmt/annot', namespaces=ns)

            # Ensure notesStmt and annot exist in the target workList if not already created by CSV data
            if target_annot is None:
                 notes_stmt = work_list.find('./notesStmt', namespaces=ns)
                 if notes_stmt is None:
                      notes_stmt = etree.SubElement(work_list, "notesStmt")
                      notes_stmt.set(xml_ns + 'id', generate_xml_id())
                 target_annot = etree.SubElement(notes_stmt, "annot")
                 target_annot.set(xml_ns + 'id', generate_xml_id())
                 target_annot.text = "" # Initialize with empty text


            # --- Consolidate annotation text from subsequent meiHeads into the target annot ---
            annotation_texts_to_merge = []
            all_mei_heads_for_merge = root.findall('.//meiHead', namespaces=ns) # Re-find all heads

            # Iterate through all meiHeads (except the target) to find annotations
            for extra_mei_head in all_mei_heads_for_merge:
                 if extra_mei_head is not None and extra_mei_head != mei_head: # Exclude the target meiHead
                     extra_annot = extra_mei_head.find('.//notesStmt/annot', namespaces=ns)
                     if extra_annot is not None and extra_annot.text:
                          annotation_texts_to_merge.append(extra_annot.text.strip())
                          print(f"  Extracted annotation text from extra meiHead for annotation consolidation in {os.path.basename(mei_file_path)}")


            # Append the collected annotation texts to the target annot
            if annotation_texts_to_merge:
                 existing_annot_text = target_annot.text.strip() if target_annot.text else ""
                 new_annot_text = existing_annot_text
                 if new_annot_text:
                     new_annot_text += " ; " # Add separator if existing text is present
                 new_annot_text += " ; ".join(annotation_texts_to_merge) # Join extracted texts with separator
                 target_annot.text = new_annot_text
                 print(f"  Consolidated annotation text in target annot.")


            # --- Add work information (from CSV) to the target workList ---
            # Clear existing children of work within workList created by CSV data (if any before adding)
            # This ensures we only add the latest work details from the dictionary.
            if work_list is not None:
                existing_work = work_list.find('./work', namespaces=ns)
                if existing_work is not None:
                     work_list.remove(existing_work)


            # Add work information if enough data is present, with xml:ids
            if work_list is not None:
                 if metadata_dict.get('Title') or metadata_dict.get('Composer_Name') or metadata_dict.get('Source_Position') or metadata_dict.get('Source_Title'):
                      work = etree.SubElement(work_list, "work")
                      work.set(xml_ns + 'id', generate_xml_id())

                      work_title = etree.SubElement(work, "title")
                      work_title.set(xml_ns + 'id', generate_xml_id())
                      work_title.text = metadata_dict.get('Title', '') # Using the main Title for work title

                      # Add composer within work, with xml:ids
                      if metadata_dict.get('Composer_Name'):
                           composer_work = etree.SubElement(work, "composer")
                           composer_work.set(xml_ns + 'id', generate_xml_id())
                           pers_name_work = etree.SubElement(composer_work, "persName")
                           pers_name_work.set(xml_ns + 'id', generate_xml_id()) # Add xml:id to persName
                           pers_name_work.text = metadata_dict.get('Composer_Name', '')
                           if metadata_dict.get('Composer_VIAF'):
                               pers_name_work.set('auth', 'VIAF')
                               pers_name_work.set('auth.uri', metadata_dict.get('Composer_VIAF', ''))

                      # Add lyricist within work (assuming no explicit field), with xml:ids
                      lyricist_work = etree.SubElement(work, "lyricist")
                      lyricist_work.set(xml_ns + 'id', generate_xml_id())
                      lyricist_work.text = "" # Placeholder

                      # notesStmt and annot are handled for consolidation above, don't recreate here based on CSV fields
                      # notes_stmt = etree.SubElement(work, "notesStmt")
                      # notes_stmt.set(xml_ns + 'id', generate_xml_id())
                      # annot = etree.SubElement(notes_stmt, "annot")
                      # annot.set(xml_ns + 'id', generate_xml_id())

                      # # Construct the annot text from metadata fields
                      # annot_text_parts = []
                      # if metadata_dict.get('Composer_Name'):
                      #      pass # Already added as <composer> element

                      # if metadata_dict.get('Source_Position'):
                      #      annot_text_parts.append(f"Source : p. {metadata_dict['Source_Position']}")
                      # if metadata_dict.get('Source_Title'):
                      #      annot_text_parts.append(metadata_dict['Source_Title'])

                      # annot.text = " ; ".join(annot_text_parts)


            # --- Remove the other meiHeads ---
            all_mei_heads_after_processing = root.findall('.//meiHead', namespaces=ns) # Re-find in case any were created/manipulated
            if root is not None:
                 for mei_head_to_remove in all_mei_heads_after_processing:
                     if mei_head_to_remove is not None and mei_head_to_remove != mei_head: # Exclude the target meiHead
                          try:
                               root.remove(mei_head_to_remove)
                               print(f"  Removed extra meiHead from {os.path.basename(mei_file_path)}")
                          except ValueError:
                               print(f"  Warning: Could not remove extra meiHead - not found in root children.")


        # Save the updated MEI file
        output_path = os.path.join(output_folder, os.path.basename(mei_file_path))
        with open(output_path, 'wb') as f:
            f.write(etree.tostring(root, pretty_print=True, encoding='utf-8', xml_declaration=True))

    except Exception as e:
        print(f"Error processing {os.path.basename(mei_file_path)}: {e}")

# Replace the original apply_metadata with the fixed version
MEI_Metadata_Updater.apply_metadata = apply_metadata_fixed

In [315]:
# Load metadata CSV from Gsheet:
metadata_csv_url = "https://docs.google.com/spreadsheets/d/1bdme5pDjX7Rolr_2dDEa5lcWJD1audjeBCyR4dyLg4s/export?format=csv&gid=1213086678"

# a dataframe from that sheet
df = pd.read_csv(metadata_csv_url).fillna('')

# a list of dictionaries from the dataframe
gesualdo_metadata_dict_list = df.to_dict(orient='records')

In [316]:
mei_paths = glob.glob('/content/drive/MyDrive/Métadonnées_Gesualdo/Fichiers_MEI_input/*')
output_folder = '/content/drive/MyDrive/Métadonnées_Gesualdo/Fichiers_MEI_output'

In [317]:
metadata_updater = MEI_Metadata_Updater()

In [318]:
pairs_to_process = []
for mei_path in mei_paths:
    mei_file_name = os.path.basename(mei_path)
    matching_dict = next((item for item in gesualdo_metadata_dict_list if item['MEI_Name'] == mei_file_name), None)
    tup = mei_path, matching_dict
    pairs_to_process.append(tup)

display(pairs_to_process)

[('/content/drive/MyDrive/Métadonnées_Gesualdo/Fichiers_MEI_input/2729_In_te_Domine_speravi_RDB.mei',
  None),
 ('/content/drive/MyDrive/Métadonnées_Gesualdo/Fichiers_MEI_input/2727_All_ombra_degli_allori_RDB.mei',
  {'RDL_ID': 2727,
   'MEI_Name': '2727_All_ombra_degli_allori_RDB.mei',
   'Title': "All'ombra degli allori",
   'Permalink': 'https://ricercardatalab.cesr.univ-tours.fr/works/2727/',
   'Composer_Name': '#265 Carlo Gesualdo',
   'Composer_VIAF': 'https://viaf.org/fr/viaf/305913142',
   'Composer_BNF_ID': 'https://data.bnf.fr/fr/ark:/12148/cb13894418z',
   'Source_Title': '#205 Ottavo libro de Madrigali, A Cinque Voci, di Pomponio Nenna Cavilier di Cesare',
   'Source_Position': '18',
   'Source_Date': '',
   'Source_Country': '',
   'Source_City': '',
   'Editor': 'Vincent Besson - Project manager | Augustin Braud - Collaborative work | Hyacinthe Belliot - Data collector | Max Blistain - Student work',
   'Editor_ORCID': 'https://orcid.org/0000-0001-5618-8602 | https:/

In [319]:
output_folder = '/content/drive/MyDrive/Métadonnées_Gesualdo/Fichiers_MEI_output'
# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)


# Assuming your CSV is loaded into a pandas DataFrame named `df_metadata`
# And the MEI files are listed in `mei_files` as defined previously

# Create an instance of the updater
metadata_updater = MEI_Metadata_Updater()

# Assuming you have a list of pairs (mei_file_path, metadata_dictionary)
# as created in the previous steps, e.g., `pairs_to_process`

# Loop through the pairs and apply metadata
for mei_file_name, matching_dict in pairs_to_process:
    # --- Debugging Print ---
    print(f"\nProcessing file: {os.path.basename(mei_file_name)}")
    print(f"Metadata dictionary for this file: {matching_dict}")
    # --- End Debugging Print ---
    metadata_updater.apply_metadata(mei_file_name, matching_dict, output_folder)

print("\nMetadata update process completed.")


Processing file: 2729_In_te_Domine_speravi_RDB.mei
Metadata dictionary for this file: None
Getting 2729_In_te_Domine_speravi_RDB.mei
Error processing 2729_In_te_Domine_speravi_RDB.mei: 'NoneType' object has no attribute 'get'

Processing file: 2727_All_ombra_degli_allori_RDB.mei
Metadata dictionary for this file: {'RDL_ID': 2727, 'MEI_Name': '2727_All_ombra_degli_allori_RDB.mei', 'Title': "All'ombra degli allori", 'Permalink': 'https://ricercardatalab.cesr.univ-tours.fr/works/2727/', 'Composer_Name': '#265 Carlo Gesualdo', 'Composer_VIAF': 'https://viaf.org/fr/viaf/305913142', 'Composer_BNF_ID': 'https://data.bnf.fr/fr/ark:/12148/cb13894418z', 'Source_Title': '#205 Ottavo libro de Madrigali, A Cinque Voci, di Pomponio Nenna Cavilier di Cesare', 'Source_Position': '18', 'Source_Date': '', 'Source_Country': '', 'Source_City': '', 'Editor': 'Vincent Besson - Project manager | Augustin Braud - Collaborative work | Hyacinthe Belliot - Data collector | Max Blistain - Student work', 'Editor_